In [39]:
import ee
import ee
import geopandas as gpd
import datetime
import folium
import geemap
import datetime
import pandas as pd

In [67]:
ee.Initialize()


shapefile_path = '/home/satyukt/Sarthak/Crop_analysis/shp/AOI_02.shp'
gdf = gpd.read_file(shapefile_path)


geojson_dict = gdf.iloc[0].geometry.__geo_interface__

aoi = ee.Geometry(geojson_dict)


sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(aoi) 
bands = ["B2", "B3", "B4", "B8", "NDVI"]
def calculateNDVI(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)



In [91]:
gdf_trash = gdf.to_crs(3857)
gdf_trash.area.values[0]

720615741.6492114

In [68]:
start_date =datetime.datetime.strptime("2022-01-01", "%Y-%m-%d")

ndvi_list = []

for i in range(48):
    
    end_date = start_date + datetime.timedelta(days=14)
    filtered = sentinel2.filterDate(start_date, end_date).map(calculateNDVI).qualityMosaic('NDVI').select(bands)
    start_date = end_date
    ndvi_list.append(filtered)

In [69]:
stack_image = ee.Image(ndvi_list[0])

for i in range(1, len(ndvi_list)):
    stack_image = stack_image.addBands(ndvi_list[i])

In [70]:
crop_gdf = gpd.read_file('/home/satyukt/Sarthak/Crop_analysis/shp/crop.shp')
crop_gdf.columns = ['id', 'Class', 'geometry']

In [109]:
water_gdf = gpd.read_file("/home/satyukt/Sarthak/Crop_analysis/shp/water.shp")
water_gdf["Class"] = 4


In [110]:
classes = pd.concat([water_gdf, crop_gdf])
# classes['geometry'] = classes['geometry'].centroid
classes

,id,geometry,Class
0,1,"POLYGON ((82.68078 17.60864, 82.68238 17.60865...",4
1,2,"POLYGON ((82.67193 17.59451, 82.66982 17.59516...",4
2,3,"POLYGON ((82.67550 17.58850, 82.67670 17.58908...",4
3,4,"POLYGON ((82.68849 17.61500, 82.68859 17.61505...",4
4,5,"POLYGON ((82.69004 17.61602, 82.69018 17.61608...",4
...,...,...,...
74,73,"POLYGON ((82.96621 17.47314, 82.96560 17.47570...",3
75,74,"POLYGON ((82.95092 17.47945, 82.95154 17.47940...",3
76,75,"POLYGON ((82.94817 17.47880, 82.94941 17.47891...",3
77,76,"POLYGON ((82.95182 17.47277, 82.95189 17.47242...",3


In [111]:
classes.to_file("/home/satyukt/Sarthak/Crop_analysis/shp/classes_poly.shp")

In [96]:
train = geemap.geopandas_to_ee(classes)

In [104]:
classestraining = stack_image.sampleRegions(
    collection=train,
    properties=['Class'],
    scale=10
)
trained_classifier = ee.Classifier.smileRandomForest(100).train(
    features=classestraining,
    classProperty="Class",
    inputProperties=stack_image.bandNames(),
)
classified = stack_image.classify(trained_classifier).toInt()


In [80]:
prop = training.first().getInfo()["properties"]
prop["Class"]

4

In [82]:
Map = geemap.Map()

In [107]:
geemap.ee_export_image(classified,
                       filename="/home/satyukt/Desktop/classified.tif",
                       scale=10,
                       crs="EPSG:4326",
                       region=aoi)

Generating URL ...
Please wait ...
An error occurred while downloading.
